### Construção de um grafo da cidade de São Paulo

Este notebook constrói um grafo (objeto da biblioteca networkx) que modela as ruas da cidade de São Paulo.

As arestas modelam os logradouros (ruas, avenidas) e os vértices são as conexões entre eles (esquinas, cruzamentos).

O grafo será utilizado para armazenar os eventos estudados (acidentes e congestionamentos)

In [1]:
import geopandas as gpd
from pyproj import CRS
import networkx as nx
import aux.sao_paulo_shapefile as sp_shp
import aux.sao_paulo_graph as sp_graph

# se nenessário recarregar os arquivos auxiliares
# from importlib import reload
# reload(sp_shp)

##### Leitura e tratamento do arquivo de logradouros (shapefile)

In [2]:
# arquivos com todos os logradouros (ruas, avenidas, ...) e suas coordenadas
gdf_logradouros = sp_shp.read_file('data/logradouros/SIRGAS_SHP_logradouronbl_line.shp')
# 222.532 registros

In [3]:
# tratamento de nulos
#gdf_logradouros = sp_shp.process_null_values(gdf_logradouros)
# 178564 registros não nulos

In [4]:
# regulariza colunas (comprimento e nome)
gdf_logradouros = sp_shp.standardize_columns(gdf_logradouros)

##### Construção do grafo

Os códigos para construção foram colocados no arquivo `aux/sao_paulo_graph.py`.

A ideia consiste em unir os começos e fins das ruas em vértices com uma tolerância de 5m (parâmetro `min_distance`).

In [5]:
G = sp_graph.create_graph (gdf_logradouros, print_progress = False)

print('Número de vértices:    ', G.number_of_nodes())
print('Número de arestas:     ', G.number_of_edges())
print('Número de componentes: ', nx.number_connected_components(G))

Número de vértices:     185665
Número de arestas:      222532
Número de componentes:  11181


In [ ]:
sp_graph.save_in_html_map(G, 'mapas/mapa_vertices_1.html')

In [6]:
# combinando vértices muito próximos (vindo de imprecisões das coordenadas do shapefile)

min_distance = 5e-5 # 5 metros no sistema de coordenadas let/lon
G = sp_graph.combine_close_nodes (G, min_distance = 5e-5, print_progress = False)

print('Número de vértices:    ', G.number_of_nodes())
print('Número de arestas:     ', G.number_of_edges())
print('Número de componentes: ',  nx.number_connected_components(G))

Número de vértices:     156156
Número de arestas:      222532
Número de componentes:  4458


In [ ]:
sp_graph.save_in_html_map(G, 'mapas/mapa_vertices_2.html')

In [7]:
G = sp_graph.remove_connection_nodes (G, print_progress = False)

print('Número de vértices:    ', G.number_of_nodes())
print('Número de arestas:     ', G.number_of_edges())
print('Número de componentes: ',  nx.number_connected_components(G))

Número de vértices:     143008
Número de arestas:      208819
Número de componentes:  4458


In [8]:
sp_graph.save_in_html_map(G, 'mapas/mapa_vertices_3.html')

#### Análises do Grafo

##### Conectividade

In [6]:
n = nx.number_connected_components(G)
print('Há', n, 'componentes conexos no grafo') # 1106 

Há 11181 componentes conexos no grafo


In [7]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
S = [(g, g.number_of_nodes(), g.number_of_edges()) for g in S]
S.sort(key = lambda s : s[1] + s[2], reverse = True)

In [8]:
n = G.number_of_nodes() + G.number_of_edges()
for i in range(5):
    s = S[i]
    v = s[1]
    a = s[2]
    p = round((v + a) / n * 100, 1)
    print('Componente {}:\t\t {} nós\t {} arestas\t\t {} % do grafo'.format(
        i, str(v).rjust(5, ' '), str(a).rjust(6, ' '),
        str(p).rjust(4, ' ')))
          
v = sum([s[1] for s in S[i:]])
a = sum([s[2] for s in S[i:]])
p = round((v + a) / n * 100, 1)
print('Demais componentes:\t {} nós\t {} arestas\t\t {} % do grafo'.format(
       str(v).rjust(5, ' '), str(a).rjust(6, ' '), str(p).rjust(4, ' ')))

Componente 0:		 140224 nós	 181889 arestas		 78.9 % do grafo
Componente 1:		  1210 nós	   1447 arestas		  0.7 % do grafo
Componente 2:		  1114 nós	   1360 arestas		  0.6 % do grafo
Componente 3:		   893 nós	   1057 arestas		  0.5 % do grafo
Componente 4:		   687 nós	    770 arestas		  0.4 % do grafo
Demais componentes:	 42224 nós	  36779 arestas		 19.4 % do grafo


In [13]:
sp_graph.save_in_html_map(S[0][0], 'mapas/mapa_vertices_componente0.html')
sp_graph.save_in_html_map(S[1][0], 'mapas/mapa_vertices_componente1.html')
sp_graph.save_in_html_map(S[2][0], 'mapas/mapa_vertices_componente2.html')
sp_graph.save_in_html_map(S[3][0], 'mapas/mapa_vertices_componente3.html')
sp_graph.save_in_html_map(S[20][0], 'mapas/mapa_vertices_componente20.html')

#### Análise do maior componente

In [94]:
H = S[0][0]

In [ ]:
diametro = nx.diameter(H)
print(diametro)
print('Diâmetro do grafo: ', diametro / 1000, 'km')

1


In [ ]:
# distribuição dos graus
# distribuição do comprimento das arestas